In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install timm torch torchvision


In [5]:
import os
import torch
import torchvision
import timm
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split


In [8]:
!pip install kaggle



In [9]:
import os
print(os.listdir("/kaggle/input/"))


['mango-leaf']


In [10]:
import os
print(os.listdir("/kaggle/input/mango-leaf/"))  # Check files inside the folder


['MangoLeafBD_Without_Testset_Augmentation']


In [12]:
import os

# Check working directory
print(os.listdir("/kaggle/working/"))

# Check extracted dataset folder
print(os.listdir("/kaggle/input/mango-leaf/"))  


['.virtual_documents', 'checkpoints']
['MangoLeafBD_Without_Testset_Augmentation']


In [13]:
import os

# Correct dataset path
dataset_path = "/kaggle/input/mango-leaf/MangoLeafBD_Without_Testset_Augmentation"

# Check if Train and Test folders exist
print("Contents of dataset:", os.listdir(dataset_path))  


Contents of dataset: ['Test', 'Train']


In [14]:
train_dir = os.path.join(dataset_path, "Train")  
test_dir = os.path.join(dataset_path, "Test")

print("Training Data:", os.listdir(train_dir))  
print("Test Data:", os.listdir(test_dir))  


Training Data: ['Powdery Mildew', 'Cutting Weevil', 'Anthracnose', 'Bacterial Canker', 'Sooty Mould', 'Gall Midge', 'Healthy', 'Die Back']
Test Data: ['Scooty Mould', 'Powdary Mildew', 'Cutting Weevil', 'Anthracnose', 'Bacterial Canker', 'Gall Midge', 'Healthy', 'Die Back']


In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resizing images to 224x224 for ViT
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [16]:
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_classes = len(train_dataset.classes)  # Get the number of classes
print(f"Number of classes: {num_classes}")


Number of classes: 8


In [17]:
class ViTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(ViTClassifier, self).__init__()
        self.model = timm.create_model("vit_base_patch16_224", pretrained=True)
        self.model.head = nn.Linear(self.model.head.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTClassifier(num_classes=num_classes).to(device)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)


In [23]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# Define dataset paths
train_dir = os.path.join(dataset_path, "Train")  
test_dir = os.path.join(dataset_path, "Test")

# Define image transformations
from torchvision import transforms
import torch.nn as nn

class CustomViT(nn.Module):
    def __init__(self, original_vit):
        super().__init__()
        self.vit = original_vit
        self.dropout = nn.Dropout(0.3)  # Apply dropout
        self.fc = nn.Linear(1000, num_classes)  # Adjust last layer

    def forward(self, x):
        x = self.vit(x)
        x = self.dropout(x)  
        return self.fc(x)

transform = transforms.Compose([
    
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1,1]


])

# Load datasets
train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print("Train DataLoader initialized successfully!")


Train DataLoader initialized successfully!


In [24]:
# Directory to save model checkpoints
checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)  # Create directory if it doesn't exist

In [25]:
import os

checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)  # Create directory if it doesn't exist

print("Checkpoint Directory Exists:", os.path.exists(checkpoint_dir))
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))


Checkpoint Directory Exists: True
Files in /kaggle/working/: ['.virtual_documents', 'checkpoints']


In [29]:
import os

checkpoint_dir = "/kaggle/working/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)  # Create directory if it doesn't exist

print("Checkpoint Directory Exists:", os.path.exists(checkpoint_dir))
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))

Checkpoint Directory Exists: True
Files in /kaggle/working/: ['.virtual_documents', 'checkpoints', 'checkpoints1']


In [32]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [30]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(train_dataset.classes)

num_classes = len(train_dataset.classes)

model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=num_classes,
    ignore_mismatched_sizes=True  # ✅ Fix size mismatch
)
model.to(device)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([8, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [35]:
riterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

In [65]:
import torch
import torch.nn as nn

def train(model, train_loader, criterion, optimizer, device):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs.logits, labels)  # Compute loss

        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"🟢 Training Loss: {avg_loss:.4f}")
    
    return avg_loss


In [108]:
def save_checkpoint(epoch, model, optimizer, loss):
    """Save model checkpoint every 5 epochs."""
    checkpoint_path = f"/kaggle/working/vit_mango_epoch{epoch}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"✅ Checkpoint saved at: {checkpoint_path}")

# Training loop
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, train_loader, criterion, optimizer, device)  # Replace with your training function

    # Save a checkpoint every 5 epochs
    if epoch % 5 == 0:
        save_checkpoint(epoch, model, optimizer, train_loss)



🟢 Training Loss: 0.0180
🟢 Training Loss: 0.0096
🟢 Training Loss: 0.0136
🟢 Training Loss: 0.0151
🟢 Training Loss: 0.0147
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch5.pth
🟢 Training Loss: 0.0122
🟢 Training Loss: 0.0176
🟢 Training Loss: 0.0208
🟢 Training Loss: 0.0116
🟢 Training Loss: 0.0179
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch10.pth
🟢 Training Loss: 0.0102
🟢 Training Loss: 0.0151
🟢 Training Loss: 0.0125
🟢 Training Loss: 0.0154
🟢 Training Loss: 0.0164
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch15.pth
🟢 Training Loss: 0.0145
🟢 Training Loss: 0.0143
🟢 Training Loss: 0.0164
🟢 Training Loss: 0.0214
🟢 Training Loss: 0.0131
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch20.pth
🟢 Training Loss: 0.0123
🟢 Training Loss: 0.0157
🟢 Training Loss: 0.0083
🟢 Training Loss: 0.0124
🟢 Training Loss: 0.0151
✅ Checkpoint saved at: /kaggle/working/vit_mango_epoch25.pth
🟢 Training Loss: 0.0163
🟢 Training Loss: 0.0159
🟢 Training Loss: 0.0254
🟢 Training Loss: 0.0116


In [39]:
# Save checkpoint only every 5 epochs
if (epoch + 1) % 5 == 0:
    save_checkpoint(epoch + 1, model, optimizer, avg_loss)


In [40]:
def save_checkpoint(epoch, model, optimizer, loss):
    checkpoint_path = "/kaggle/working/vit_mango_latest.pth"  # Always overwrite
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }, checkpoint_path)
    print(f"Checkpoint saved at Epoch {epoch} ✅")


In [96]:
!df -h


Filesystem                                                               Size  Used Avail Use% Mounted on
overlay                                                                  7.9T  6.1T  1.9T  77% /
tmpfs                                                                     64M     0   64M   0% /dev
shm                                                                       14G  4.0K   14G   1% /dev/shm
/dev/sdb1                                                                122G   25G   97G  21% /opt/bin
/dev/loop1                                                                20G   19G  768M  97% /kaggle/lib
192.168.13.2:/data/kagglesdsdata/datasets/6978863/11180905/d8qsjcxm5liv   50T   35T   16T  69% /kaggle/input/mango-leaf
/dev/mapper/snap                                                         7.9T  6.1T  1.9T  77% /etc/hosts
tmpfs                                                                     16G     0   16G   0% /proc/acpi
tmpfs                                             

In [97]:
!rm -rf /kaggle/working/*.pth  # Delete all old checkpoints
!rm -rf /kaggle/working/__pycache__/  # Remove Python cache files


In [67]:
import torch

def save_checkpoint(epoch, model, optimizer, loss):
    checkpoint_path = f"/kaggle/working/vit_mango_epoch{epoch}.pth"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"✅ Checkpoint saved at: {checkpoint_path}")


In [109]:
from IPython.display import FileLink

# Change the filename to your last saved checkpoint
checkpoint_filename = "vit_mango_epoch50.pth"
checkpoint_path = f"/kaggle/working/{checkpoint_filename}"

# Generate a download link
FileLink(checkpoint_path)


/kaggle/working/vit_mango_epoch50.pth

In [110]:
!zip -r /kaggle/working/model_checkpoints.zip /kaggle/working/vit_mango_epoch*.pth


  adding: kaggle/working/vit_mango_epoch10.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch15.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch20.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch25.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch30.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch35.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch40.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch45.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch50.pth (deflated 8%)
  adding: kaggle/working/vit_mango_epoch5.pth (deflated 8%)


In [111]:
from IPython.display import FileLink

FileLink("/kaggle/working/model_checkpoints.zip")


/kaggle/working/model_checkpoints.zip

In [94]:
!ls -lh /kaggle/working/


total 12K
drwxr-xr-x 2 root root 4.0K Mar 27 08:54 checkpoints
-rw-r--r-- 1 root root   22 Mar 27 09:15 checkpoints.zip
-rw-r--r-- 1 root root   22 Mar 27 09:16 model_checkpoints.zip.zip


In [95]:
!ls -lh /kaggle/working/checkpoints

total 19G
-rw-r--r-- 1 root root 8.9G Mar 27 08:54 model_checkpoints.zip
-rw-r--r-- 1 root root 983M Mar 27 07:43 vit_mango_epoch10.pth
-rw-r--r-- 1 root root 983M Mar 27 07:50 vit_mango_epoch15.pth
-rw-r--r-- 1 root root 983M Mar 27 07:56 vit_mango_epoch20.pth
-rw-r--r-- 1 root root 983M Mar 27 08:03 vit_mango_epoch25.pth
-rw-r--r-- 1 root root 983M Mar 27 08:09 vit_mango_epoch30.pth
-rw-r--r-- 1 root root 983M Mar 27 08:16 vit_mango_epoch35.pth
-rw-r--r-- 1 root root 983M Mar 27 08:22 vit_mango_epoch40.pth
-rw-r--r-- 1 root root 983M Mar 27 08:29 vit_mango_epoch45.pth
-rw-r--r-- 1 root root 983M Mar 27 08:35 vit_mango_epoch50.pth
-rw-r--r-- 1 root root 983M Mar 27 07:37 vit_mango_epoch5.pth


In [78]:
!pip install kaggle

In [81]:
!kaggle datasets init -p /kaggle/working/

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [92]:
import IPython
from IPython.display import Javascript

def download(file_path):
    file_name = file_path.split('/')[-1]
    jscode = f"""
    var link = document.createElement('a');
    link.href = '{file_path}';
    link.download = '{file_name}';
    document.body.appendChild(link);
    link.click();
    document.body.removeChild(link);
    """
    IPython.display.display(Javascript(jscode))

# Call the function with the path to the ZIP file
download('/kaggle/working/checkpoints/model_checkpoints.zip')


<IPython.core.display.Javascript object>

In [112]:
from IPython.display import FileLink
FileLink('/kaggle/working/model_checkpoints.zip')

/kaggle/working/model_checkpoints.zip